In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr’

%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append('/Users/siyuyang/Source/repos/GitHub_MSFT/CameraTraps')

In [22]:
import os
import json
import urllib

from tqdm import tqdm

from data_management.megadb import sequences_schema_check
from data_management.annotations.add_bounding_boxes_to_megadb import extract_annotations, add_annotations_to_sequences, zsl_image_filename_map_func, default_image_filename_map_func

In [4]:
out_dir = '/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/megadb_batches_9_10_11'

## ZSL
Need to regenerate - confirmed empty info not captured

In [23]:
batch_9 = '/Users/siyuyang/Source/temp_data/CameraTrap/annotations/201910/MsWLB_imerit_batch_9_all_3sep2019'

In [38]:
image_filename_to_bboxes = extract_annotations(batch_9, 'zslborneo')

  1%|          | 931/99485 [00:00<00:21, 4648.73it/s]

5 files found in directory at annotation_path
Number of annotation entries found: 99485


100%|██████████| 99485/99485 [00:22<00:00, 4512.53it/s]

102120 boxes on 80552 images were in the annotation file(s). 0 boxes skipped because they are not for the requested dataset

Category counts for the bboxes:
animal: 62036
person: 37873
vehicle: 2211


In [25]:
image_filename_to_bboxes['3b6a063a-9910-11e9-98e8-1c872c5fa89b.jpg']

[{'bbox': [0.4116, 0.3887, 0.1455, 0.1028], 'category': 'animal'}]

In [26]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/megadb/zsl_borneo.json') as f:
    sequences = json.load(f)

In [27]:
sequences, images_updated = add_annotations_to_sequences(sequences, image_filename_to_bboxes, zsl_image_filename_map_func)

Dataset to which the sequences belong to: zsl_borneo. Make sure this is the intended dataset where the bboxes also come from!
80348 images updated; 0 images had their bbox overwritten; 423645 images not updated


In [28]:
80552 - 80348

204

In [29]:
images_updated_set = set(images_updated)

In [30]:
images_w_anno = set(image_filename_to_bboxes.keys())

In [31]:
diff = images_w_anno - images_updated_set
len(diff)

204

This is fewer than the 313 images that were left out from the sequence entries because they were grouped into sequences incorrectly, so possible that these were among the ones left out

In [35]:
with open(os.path.join(out_dir, 'zsl_borneo_w_batch_9_boxes.json'), 'w') as f:
    json.dump(sequences, f)

In [36]:
with open(os.path.join(out_dir, 'zsl_borneo_w_batch_9_boxes_indented.json'), 'w') as f:
    json.dump(sequences, f, indent=1)

## WPS

In [5]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/megadb/wps_190624.json') as f:
    wps_sequences = json.load(f)

In [6]:
batch_10 = '/Users/siyuyang/Source/temp_data/CameraTrap/annotations/201910/batch10'

In [15]:
wps_image_filename_to_bboxes = extract_annotations(batch_10, 'wps_190624')

  1%|▏         | 2327/177122 [00:00<00:07, 23249.51it/s]

19 files found in directory at annotation_path
Number of annotation entries found: 177122


100%|██████████| 177122/177122 [00:16<00:00, 11041.20it/s]

50386 boxes on 30000 images were in the annotation file(s). 165383 boxes skipped because they are not for the requested dataset

Category counts for the bboxes:
animal: 6350
group: 60
person: 32346
vehicle: 11630


In [16]:
len(wps_image_filename_to_bboxes)

30000

In [17]:
keys = list(wps_image_filename_to_bboxes.keys())
keys[0]

'21b4d713-1c04-45a2-a12e-1a48565ab401.jpg'

In [18]:
wps_image_filename_to_bboxes['21b4d713-1c04-45a2-a12e-1a48565ab401.jpg']

[{'bbox': [0.7293, 0.5116, 0.1178, 0.3378], 'category': 'person'}]

In [19]:
wps_sequences, wps_images_updated = add_annotations_to_sequences(wps_sequences, wps_image_filename_to_bboxes, default_image_filename_map_func)

Dataset to which the sequences belong to: wps_190624. Make sure this is the intended dataset where the bboxes also come from!
30000 images updated; 0 images had their bbox overwritten; 58489 images not updated


In [21]:
# ! Also need to add __label_unavailable to the image class field. These came to us unlabeled
for seq in wps_sequences:
    for im in seq['images']:
        im['class'] = ['__label_unavailable']

In [24]:
sequences_schema_check.sequences_schema_check(wps_sequences)

Verified that the sequence items conform to the schema.
Verified that the sequence items meet the additional requirements.


In [25]:
with open(os.path.join(out_dir, 'wps_190624_w_batch_10_boxes.json'), 'w') as f:
    json.dump(wps_sequences, f, indent=1)